In [16]:
import pandas as pd
import gspread

from datetime import date
from datetime import datetime
from datetime import datetime, timedelta

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import zoho_crm_api_module as crm
import zoho_campaign_api_module as campaign
import requests

import sys
sys.path.append('../../../zoho_data_app/')

# importing
import main_module as m
import zoho_filter_module as fil


gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar\Documents\Python\Data\Google Credentials\fajar-fatoni-OAuth.json'
)

SHEET_ID = '1OePtqWqyjzNwA7Jmwg6IU7fcveoV_qS7hd_nBeOW3zE'
SHEET_NAME = 'Form Responses 1'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
def next_available_row(worksheet):
    str_list = worksheet.col_values(1)
    return str(len(str_list)+1)

# mencari tanggal 8 hari kebelekang
calculate_date = datetime.now() - timedelta(days=8)
new_cal_date = str(calculate_date.strftime("%Y-%m-%d")).split(' ')
first_date = new_cal_date[0]
# first_date = '2022-12-06'

# mencari tanggal hari ini
today = date.today()
second_date = today.strftime("%Y-%m-%d")
# second_date = '2022-12-14'

start_date = '\''+first_date+'T00:00:01+08:00\''
end_date = '\''+second_date+'T23:59:59+08:00\''

In [18]:
# get data gsheet

spreadsheet = gc.open_by_key(SHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)
rows = worksheet.get_all_records()

# print(rows[:5])

# print('==============================')
df_gsheets = pd.DataFrame(rows)
df_gsheets

GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [ ]:
df_gsheets['Timestamp'] = df_gsheets['Timestamp'].astype('datetime64[ns]')
df_gsheets['Email Address'] = df_gsheets['Email Address'].astype('string')
df_gsheets['Phone Number'] = df_gsheets['Phone Number'].astype('string')
df_gsheets

,Timestamp,Email Address Blank,Firstname,Lastname,Email Address,Phone Number,Address,Date of Birth,Do you live in Bali,Hostess
0,2023-12-09 08:58:26,,Agus,Maryana,agus.maryana@karmagroup.com,628122730101,Denpasar,5/14/1982,Yes,Bagus
1,2023-12-11 17:31:21,,Kiki,Kiki,andrianikiki@gmail.com,,Jakarta,,No,Darmi
2,2023-12-11 17:33:51,,Karina,Karina,karinavelikaya31@gmail.com,380934470834,Sitadiens hotel,,No,Darmi
3,2023-12-12 12:30:35,,Bakker,Bakker,stbakker@hitmail.com,31628896338,Villa shato,,No,Efa
4,2023-12-12 12:37:06,,Carmila,Carmila,jcarmela@gmail.com,-,Jimbaran,,Yes,Efa
...,...,...,...,...,...,...,...,...,...,...
416,2024-01-04 16:13:22,,Sabine,,sabine_dante@mail.ru,79037783399,,,No,Krisma
417,2024-01-04 16:16:10,,Vegard,,vaahre28@hotmail.com,4740937794,,,No,Krisma
418,2024-01-04 16:17:23,,Nina,,missis.ninettak2021@yandex.ru,79167830870,,,No,Krisma
419,2024-01-04 16:19:16,,Lisa,,lisakulinchenko@mail.ru,79165659640,,,No,Krisma


In [ ]:
# df_gsheets['Timestamp'].loc[df_gsheets['Timestamp'] > new_cal_date]

ValueError: ('Lengths must match to compare', (421,), (1,))

In [ ]:
df_gsheets.dtypes

Timestamp              string
Email Address Blank    string
Firstname              string
Lastname               string
Email Address          string
Phone Number           string
Address                string
Date of Birth          string
Do you live in Bali    string
Hostess                string
dtype: object

In [ ]:
campaign_token = campaign.get_campaign_token()
crm_token = crm.get_crm_token()

cols = [
    'Created_Time', 
    'id', 
    'First_Name', 
    'Last_Name', 
    'Email', 
    'Phone', 
    'Mobile', 
    'Brand', 
    'Lead_Sub_Brand', 
    'Lead_Source', 
    'Lead_Regions',
    'Lead_Locations', 
    'City', 
    'Nationality', 
    'Country', 
    'Booking_Status', 
    'Guest_Type', 
    'Email_Opt_In',
    'Email_Opt_Out', 
    'BookRef',
    'Email_status', 
    'Odyssey_Members', 
    'Do_Not_Mail', 
    'Do_Not_Contact'
]

Timestamp              string
Email Address Blank    string
Firstname              string
Lastname               string
Email Address          string
Phone Number           string
Address                string
Date of Birth          string
Do you live in Bali    string
Hostess                string
dtype: object

In [ ]:
email = 'asda'
phone = '1231sad'

df_gsheets["Check Exists"] = df_gsheets.apply(
    lambda x: crm.check_karma_beach_exists(crm_token, cols, x['Email Address'], x['Phone Number']),
    axis = 1
)

KeyboardInterrupt: 

In [ ]:
df_gsheets

In [ ]:
m.clean_number(df_gsheets, 'Phone Number')
m.clean_number(df, 'Phone')
m.replace_not_valid_to_empty(df_gsheets, 'Phone Number')
m.replace_not_valid_to_empty(df, 'Phone')
df_gsheets['Email Address'] = m.lowercase(df_gsheets, 'Email Address')
m.clean_space(df_gsheets, 'Email Address')
df['Email'] = m.lowercase(df, 'Email')
m.clean_space(df, 'Email')

0          rikki.ding@hotmail.com
1          baracuda.art@gmail.com
2       karyna_podosyan@yahoo.com
3               ihanapc@gmail.com
4             v_bk@rediffmail.com
                  ...            
166         pinar-tas@outlook.com
167          arzumsenel@gmail.com
168            valueva777@mail.ru
169    usacevaevgenia34@gmail.com
170                rahel91@gmx.de
Name: Email, Length: 171, dtype: object

In [ ]:
# df = df.reindex(columns=['Account ID', 'ContactID', 'FirstName', 'LastName',
#        'AddressLine1', 'AddressLine2', 'Town', 'City', 'Country', 'Post Code',
#        'Phones', 'Owner', 'Family', 'Nationality', 'DoB', 'WeddingDate',
#        'Keywords', 'NoEDM', 'Account Status', 'Account Type', 'OdysseyNo',
#        'Servicing Office', 'Email','Category'])

In [ ]:
# df.sort_values(['Account ID', 'DoB'], ascending=True, inplace=True)
# df_clean = df.copy()

In [ ]:
# # hitung jumlah duplicate
# count_duplicate_general = df_clean.loc[df_clean.duplicated(subset=['Account ID'])].shape[0]

# df_clean.loc[df_clean.duplicated(subset=['Account ID'])]

,Account ID,ContactID,FirstName,LastName,AddressLine1,AddressLine2,Town,City,Country,Post Code,...,DoB,WeddingDate,Keywords,NoEDM,Account Status,Account Type,OdysseyNo,Servicing Office,Email,Category
36578,14,814558,Akarsh,Sudarshan,"#368, 14TH CROSS, SADASHIVANAGAR,",NaN,BANGALORE,KARNATAKA,India,560080,...,1989-02-19,NaN,NaN,NaN,Active,KARMA CLUB,C82001700,KARMA CONCIERGE INDIA,15.akarsh@gmail.com,KCI
36581,14,814558,Akarsh,Sudarshan,"#368, 14TH CROSS, SADASHIVANAGAR,",NaN,BANGALORE,KARNATAKA,India,560080,...,1989-02-19,NaN,NaN,NaN,Active,KARMA CLUB,C82001700,KARMA CONCIERGE INDIA,15.akarsh@gmail.com,KCI
36580,14,842834,Roma,Ganesh,"#368, 14TH CROSS, SUDASHIVANAGAR,",NaN,BANGALORE,KARNATAKA,India,560080,...,1989-10-20,NaN,NaN,NaN,Active,KARMA CLUB,C82001700,KARMA CONCIERGE INDIA,roma.ganesh@gmail.com,KCI
38497,25,27707,V. S.,Jayashankar,KALPAVRIKSHA # 13,BRUNTON CROSS ROAD,BANGALORE,KA,India,560 025,...,NaN,NaN,NaN,NaN,Active,WEEKS MEMBER,C82020365,KARMA CONCIERGE INDIA,pvjayashankar@hotmail.com,KCI
38887,31,34238,Padmini,Nagachandra,"NO.1628, 8TH MAIN, 3RD BLOCK",JAYANAGAR,BANGALORE,KA,India,560 011,...,NaN,NaN,NaN,NaN,Active / Open,FRACTIONAL,D89100419,KARMA CONCIERGE INDIA,sknagachandra@gmail.com,KCI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,1110264,1092118,Michel,Dumont,17440 RUE DU GRAND PRIX,-,MIRABEL,QUEBEC,Canada,J7J 2M3,...,1958-03-14,NaN,NaN,NaN,Open Deal Member,KARMA CLUB,NaN,KARMA CONCIERGE BALI,rodupomi@hotmail.ca,KCB - Non Indonesia
17834,1110268,1092130,Jocelyn,Luansing,278 W 6TH STREET,-,PITTSBURG,CALIFORNIA,USA,94565,...,1988-06-30,NaN,NaN,NaN,Open Deal Member,KARMA CLUB,NaN,KARMA CONCIERGE BALI,jbluansing@gmail.com,KCB - Non Indonesia
4076,1110270,1092133,Stephen,Grimstone,19 ELANA COURT,CROYDON,MELBOURNE,VICTORIA,Australia,3136,...,1952-06-21,NaN,NaN,NaN,Active,KARMA CLUB,C82050436,KARMA CONCIERGE BALI,stephengrimstone@henkel.com,KCB - Non Indonesia
60960,1110284,1092154,Paramananda,Ponnaiyan,B-906 MAANGALYA ASHIRWAD OPP. KOLI FARM GATE,BANNERGHATTA RD,BANGALORE,KARNATAKA,India,560083,...,1980-07-13,NaN,NaN,NaN,Open Deal Member,KARMA CLUB,NaN,KARMA CONCIERGE INDIA,palukkhanna@yahoo.com,KCI


In [ ]:
# df_clean.drop_duplicates(subset=['Account ID'], inplace=True)
# df_clean.reset_index(drop=True, inplace=True)

# # hitung jumlah setelah value duplicate dihapus
# count_after_dedup_duplicate = df_clean.shape[0]

# df_clean

,Account ID,ContactID,FirstName,LastName,AddressLine1,AddressLine2,Town,City,Country,Post Code,...,DoB,WeddingDate,Keywords,NoEDM,Account Status,Account Type,OdysseyNo,Servicing Office,Email,Category
0,5,54901,K,Ankaiyan,"NO. 1671, 9TH MAIN ROAD",4TH CROSS HAL III STAGE,BANGALORE,KA,India,560 075,...,NaN,NaN,NaN,NaN,Active,WEEKS MEMBER,C82034750,KARMA CONCIERGE INDIA,k.ankaiyan@gmail.com,KCI
1,14,842833,Leela,Sudarshan,-,NaN,-,-,India,-,...,1969-03-03,NaN,NaN,NaN,Active,KARMA CLUB,C82001700,KARMA CONCIERGE INDIA,15.akarsh@gmail.com,KCI
2,20,23850,.,Govindarajan,"VI - 75/3, GURKHA CAMP","WELLINGTON, NILGIRIS",COONOOR,TN,India,643231,...,NaN,NaN,NaN,NaN,Reminder Notice Sent,WEEKS MEMBER,C82003626,KARMA CONCIERGE INDIA,govind30008@yahoo.co.in,KCI
3,25,27707,V. S.,Jayashankar,KALPAVRIKSHA # 13,BRUNTON CROSS ROAD,BANGALORE,KA,India,560 025,...,NaN,NaN,NaN,NaN,Active,WEEKS MEMBER,C82020365,KARMA CONCIERGE INDIA,viji1997@hotmail.com,KCI
4,31,34238,Padmini,Nagachandra,"NO.1628, 8TH MAIN, 3RD BLOCK",JAYANAGAR,BANGALORE,KA,India,560 011,...,NaN,NaN,NaN,NaN,Active / Open,FRACTIONAL,D89100419,KARMA CONCIERGE INDIA,padmini.nagachandra@gmail.com,KCI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31906,1110279,1092148,Shiju,Thomas,"EA-802 , PURVA ETERNITY THENGODE P O",KAKKANAD,COCHN,KERALA,India,682030,...,1974-08-02,NaN,NaN,NaN,Open Deal Member,KARMA CLUB,NaN,KARMA CONCIERGE INDIA,shiju74@icloud.com,KCI
31907,1110284,1092155,Paluk,Khanna,B-906 MAANGALYA ASHIRWAD OPP. KOLI FARM GATE,BANNERGHATTA RD,BANGALORE,KARNATAKA,India,560083,...,1978-06-08,NaN,NaN,NaN,Open Deal Member,KARMA CLUB,NaN,KARMA CONCIERGE INDIA,param.ponnaiyan@gmail.com,KCI
31908,1110285,1092156,Sonal Kunal,Sheth,"C - 204, RAJ HEIGHTS CHS LTD","OPP VORA COLONY, M. G. ROAD, KANDIVALI (WEST)",MUMBAI,MAHARASHTRA,India,400067,...,1982-11-05,NaN,NaN,NaN,Open Deal Member,KARMA CLUB,NaN,KARMA CONCIERGE INDIA,shethkunal005@gmail.com,KCI
31909,1110297,1092180,Alpeshkumar Arvindbhai,Kanthariya,"16 / 1, SABRIDHAM,","BHIMPOR DUMAS, CHORASI,",SURAT,GUJARAT,India,394550,...,1989-06-02,NaN,NaN,NaN,Open Deal Member,KARMA CLUB,NaN,KARMA CONCIERGE INDIA,alpeshkanthariya41@gmail.com,KCI


In [ ]:
# df.loc[df['Account ID'] == 15345]
# df_clean.loc[df_clean['Account ID'] == 15345]

In [ ]:
# df_clean.sort_values(['Category', 'Account ID', 'DoB'], ascending=True, inplace=True)

In [ ]:
# df_test_input = df_clean.iloc[0:5].copy()
# df_test_input.fillna('', inplace=True)

# df_clean.fillna('', inplace=True)
# df_clean.loc[(df_clean['DoB'] == '0000-00-00'), 'DoB'] = ''

In [ ]:
# df_clean.to_excel(r'C:\Users\fajar\Documents\Python\Data\Member Clean Database(Account ID).xlsx', index=False)

In [ ]:
# df_gsheets.loc[(df_gsheets['Category'] == 'KCI') & (df_gsheets['DoB'] == '0000-00-00')]